This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '8Z3CGmo-2zY14fcg3itT'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [241]:
# First, import the relevant modules
import requests
from collections import namedtuple
from datetime import datetime

Using Quandl API commands to extract trading data from the FSE on Zeiss medical for the year 2017.

In [258]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?&start_date=2017-01-01&end_date=2017-12-31api_key=8Z3CGmo-2zY14fcg3itT"

In [259]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
rq = requests.get(url)
Zeiss_dict = rq.json()

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [260]:
column_names = Zeiss_dict['dataset_data']['column_names']
column_names = [elem.replace(' ', '') for elem in column_names]
print(column_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'TradedVolume', 'Turnover', 'LastPriceoftheDay', 'DailyTradedUnits', 'DailyTurnover']


In [261]:
datalist = Zeiss_dict['dataset_data']['data']
print(datalist[-1])

['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]


Will be useful to convert first element in each datalist member list to a datetime object and then generate a list of named tuple objects. Each object property is addressable by the column names up above.

In [262]:
ZPobject = namedtuple('ZPObject', field_names = column_names)

In [263]:
Zeiss_Prices_2017 = [Zeiss_Price._make([datetime.strptime(elem[0], '%Y-%m-%d')] + elem[1::]) for elem in datalist]

In [264]:
Zeiss_Prices_2017[-1]

Zeiss_Price(Date=datetime.datetime(2017, 1, 2, 0, 0), Open=34.99, High=35.94, Low=34.99, Close=35.8, Change=None, TradedVolume=44700.0, Turnover=1590561.0, LastPriceoftheDay=None, DailyTradedUnits=None, DailyTurnover=None)

### Finding highest and lowest Opening prices for 2017

In [265]:
open_prices_2017 = [elem.Open for elem in Zeiss_Prices_2017 if type(elem.Open) == float]

In [266]:
min_open_price = min(open_prices_2017)
print(min_open_price)

34.0


In [267]:
max_open_price = max(open_prices_2017)
print(max_open_price)

53.11


### Finding Largest price change in one day for 2017

In [268]:
HiLo_absdifferential = [abs(elem.High - elem.Low) for elem in Zeiss_Prices_2017]

In [270]:
largest_price_change_2017 = max(HiLo_absdifferential)
print(largest_price_change_2017)

2.8100000000000023


### Find largest price change between any two consecutive trading days in 2017 (based on closing value)

In [274]:
Consec_Price_Change = [abs(Zeiss_Prices_2017[i+1].Close - Zeiss_Prices_2017[i].Close) for i in range(0, len(Zeiss_Prices_2017)-1) ]

In [284]:
Largest_ConsecDay_PriceChange = max(Consec_Price_Change)
print(Largest_ConsecDay_PriceChange)

2.559999999999995


### Average Trade Volume in 2017

In [283]:
tradevollist = [elem.TradedVolume for elem in Zeiss_Prices_2017]
avgtradevol_2017 = sum(tradevollist)/len(tradevollist)
print(avgtradevol_2017)

89124.33725490196


### Calculate Median Volume in 2017

In [293]:
def calc_median(data):
    sorted_data = sorted(data)
    m = len(sorted_data)
    if m%2 == 0:
        median = sorted_data[(m/2)-1] + sorted_data[(m/2)]
    else:
        median = sorted_data[m//2]
    return median

In [298]:
median_tradevol_2017 = calc_median(tradevollist)
print(median_tradevol_2017)

76286.0
